In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import warnings
import joblib
import random
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# 1. 환경 설정
warnings.filterwarnings('ignore')
plt.rc('font', family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# 2. 경로 및 분석 타겟 설정 (CPU)
# (주의) 본인 컴퓨터 경로에 맞게 확인해주세요 ('Dataset' 또는 'last_data')
DATA_PATH = r'D:\minwoo\project\Virtual-Build-PC\Dataset\CPU_Total'
TARGET_MODEL = "i5-13400F"

# 3. 데이터 로드 및 전처리
def load_cpu_integrated(path, target):
    all_data = []
    if not os.path.exists(path):
        print(f"❌ 오류: 경로를 찾을 수 없습니다 -> {path}")
        return None

    files = sorted([f for f in os.listdir(path) if f.endswith('.csv')])
    for f in files:
        file_path = os.path.join(path, f)
        df_tmp = None
        for enc in ['utf-8', 'cp949']:
            try: df_tmp = pd.read_csv(file_path, encoding=enc); break
            except: continue
        if df_tmp is None: continue
        
        rows = df_tmp[df_tmp['Name'].str.contains(target, na=False, case=False)]
        cols = [c for c in df_tmp.columns if re.match(r'\d{4}-\d{2}-\d{2}', c)]
        for col in cols:
            p = pd.to_numeric(rows[col].astype(str).str.replace(',', '').str.extract('(\d+)')[0], errors='coerce')
            
            # [필터] CPU는 10,000원 이하 데이터(부속품 등) 제거
            valid = p[p > 10000]
            
            if not valid.empty:
                all_data.append({'Date': col.split(' ')[0], 'Price': valid.mean()})
    
    if not all_data:
        return None

    df = pd.DataFrame(all_data).groupby('Date')['Price'].mean().reset_index()
    df['Date_dt'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date_dt')
    
    # Feature Engineering
    df['Year'] = df['Date_dt'].dt.year
    df['Month'] = df['Date_dt'].dt.month
    df['DayOfWeek'] = df['Date_dt'].dt.dayofweek
    df['DaysFromStart'] = (df['Date_dt'] - df['Date_dt'].min()).dt.days
    
    # Smoothing
    df['Price_Raw'] = df['Price']
    df['Price_Smooth'] = df['Price'].rolling(window=3, min_periods=1).mean()
    return df

df = load_cpu_integrated(DATA_PATH, TARGET_MODEL)

if df is not None and len(df) > 30:
    # 4. [EDA]
    print(f"🔍 {TARGET_MODEL} 탐색적 데이터 분석(EDA) 리포트")
    plt.figure(figsize=(20, 10))

    plt.subplot(2, 3, 1)
    sns.histplot(df['Price_Raw'], kde=True, color='skyblue')
    plt.title('주요 변수별 분포 (Price Histogram)')

    plt.subplot(2, 3, 2)
    sns.countplot(data=df, x='DayOfWeek', palette='viridis')
    plt.title('요일별 데이터 분포')

    plt.subplot(2, 3, 3)
    sns.countplot(data=df, x='Month', palette='magma')
    plt.title('월별 데이터 분포')

    plt.subplot(2, 3, 4)
    sns.heatmap(df[['Price','Year','Month','DayOfWeek','DaysFromStart']].corr(), annot=True, cmap='coolwarm', fmt=".2f")
    plt.title('변수 간 상관관계 분석')

    plt.subplot(2, 3, 5)
    sns.boxplot(x=df['Price_Raw'], color='lightcoral')
    plt.title('시세 데이터 이상치(Outlier)')

    plt.subplot(2, 3, 6)
    plt.plot(df['Date_dt'], df['Price_Raw'], alpha=0.5, label='Raw')
    plt.plot(df['Date_dt'], df['Price_Smooth'], color='red', label='Trend')
    plt.legend()
    plt.title('시세 변동 Overview')

    plt.tight_layout()
    plt.show()

    # 5. 데이터셋 준비
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df[['Price_Smooth']])
    SEQ_LENGTH = 30
    X, y = [], []
    for i in range(len(scaled_data) - SEQ_LENGTH):
        X.append(scaled_data[i:i+SEQ_LENGTH])
        y.append(scaled_data[i+SEQ_LENGTH])
    X, y = np.array(X), np.array(y)

    # 6. 모델 학습
    model = Sequential([
        Input(shape=(SEQ_LENGTH, 1)),   
        LSTM(128, return_sequences=True),
        Dropout(0.2),
        LSTM(64),
        Dense(1)
    ])

    model.compile(optimizer='adam', loss='huber')
    model.fit(X, y, epochs=50, batch_size=16, verbose=1, shuffle=False)

    # 7. 평가
    y_pred_scaled = model.predict(X, verbose=0)
    y_pred = scaler.inverse_transform(y_pred_scaled)
    y_actual = df['Price_Smooth'].values[SEQ_LENGTH:]

    mse = mean_squared_error(y_actual, y_pred)
    print("\n" + "="*30)
    print("🤖 AI 모델 성능 평가 지표 (Overview)")
    print("="*30)
    print(f"1. R2 Score: {r2_score(y_actual, y_pred):.4f}")
    print(f"2. MAE: {mean_absolute_error(y_actual, y_pred):,.0f}원")
    print(f"3. MSE: {mse:,.0f}")
    print(f"4. RMSE: {np.sqrt(mse):,.0f}원")
    print("="*30)

    # ========================================================
    # [CPU 저장 로직] models 폴더에 'cpu_' 접두사로 저장
    # ========================================================
    SAVE_DIR = "models"
    if not os.path.exists(SAVE_DIR):
        os.makedirs(SAVE_DIR)
        print(f"📂 '{SAVE_DIR}' 폴더를 새로 생성했습니다.")

    safe_name = TARGET_MODEL.replace(" ", "_")
    
    # 예: models/cpu_i5-13400F_model.h5
    model_path = os.path.join(SAVE_DIR, f"cpu_{safe_name}_model.h5")
    scaler_path = os.path.join(SAVE_DIR, f"cpu_{safe_name}_scaler.pkl")

    model.save(model_path)
    joblib.dump(scaler, scaler_path)
    
    print(f"✅ [CPU] 저장 완료:\n - {model_path}\n - {scaler_path}")

else:
    print(f"⚠️ {TARGET_MODEL}: 데이터를 찾을 수 없거나 데이터량이 부족합니다.")

❌ 오류: 경로를 찾을 수 없습니다 -> D:\minwoo\project\Virtual-Build-PC\Dataset\CPU_Total
⚠️ i5-13400F: 데이터를 찾을 수 없거나 데이터량이 부족합니다.
